In [3]:
# ! pip install pandas 
# ! pip install numpy
# ! pip install gensim.models

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim import models
from gensim.models import KeyedVectors
import ast
import re
import pickle

# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 유사 태그 출력
from heapq import nlargest
from heapq import nsmallest

# # 모델에 없는 단어들 형태소 분석 
# from konlpy.tag import Okt
# from konlpy.utils import pprint
# okt = Okt()

#### DATA

In [2]:
# 데이터 받아오기 
chart_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/chart.pickle")
genre_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/genre.pickle")
playlist_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/playlist.pickle")
id = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/playlist_id.pkl")

model_path = "C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Model/cc.ko.300.vec"

In [3]:
# 전처리
playlist_tag.rename(columns = {'발매일':'발매년도', '좋아요':'좋아요 수'}, inplace = True)

chart_tag2 = chart_tag.drop('연도', axis = 1)
playlist_tag2 = playlist_tag.drop('플리', axis = 1)
songs_tag = pd.concat([chart_tag2, genre_tag, playlist_tag2])

In [4]:
# 합치기! 

songs_tag = songs_tag[songs_tag['발매년도'] != '-']
songs_tag.loc[:, '발매년도'] = songs_tag['발매년도'].apply(lambda x: str(x)[:4] if len(str(x)) > 4 else str(x))
songs_tag.loc[:, '좋아요 수'] = songs_tag['좋아요 수'].apply(lambda x: int(str(x).replace(',','')))
songs_tag = songs_tag.astype({'발매년도':'int32', '좋아요 수':'int32'})
songs_tag = songs_tag.drop_duplicates(['제목','가수'], keep = 'first')

In [5]:
# 태그 전처리 
hashtags = id.copy()

# 해시태그 없는 플레이리스트 제외
hashtags = hashtags[hashtags['tag'].str.contains("#")]
# 리스트로 전부 가져오기
only_tags = hashtags['tag'].tolist()
# "#" 제거하기
tags_fin  = []
for tag in only_tags:
  tags_fin.append(re.sub(r"[#\[\]\'\,]","",tag).split())
tags_fin[:2]

[['여름', '더워', '시원', '청량', '플리', '케이팝', 'kpop', '키오프'],
 ['팝송', '케이팝', '매장', '카페', '식당', '브금', '노래', '휴식', '신나는', '일']]

트랙 1) 일기 태그와 유사한 플리 찾아서 출력

트랙 2) 일기 태그와 가사 태그가 유사한 노래들

음악 취향 입력
일기 태그, 가사 태그 각각

#### 시도 2 - fasttext, OOV 반영

트랙 1 결과물 - filtered_hashtag 
- 모델에 없는 태그 제거 

##### 트랙 1) 플리 태그 기반 플리 찾기

In [13]:
# 사전학습 모델 로드 
fasttext = KeyedVectors.load_word2vec_format(
    model_path,
    binary=False,  # 텍스트 형식이므로 False
    encoding='utf-8',  # UTF-8 인코딩으로 로드
    unicode_errors='ignore'  # 디코딩 에러 무시
)

In [14]:
# 기존 tags_fin 중에 모델에 없는 거는 제거
def delete_missing_tags(tags_list):
    filtered_tags = []
    for tag in tags_list:
        filtered_list = [word for word in tag if word in fasttext]
        filtered_tags.append(filtered_list)
    return filtered_tags

In [15]:
# 기존 해시태그에서 모델에 없는 단어 제외
filtered_hashtag = delete_missing_tags(tags_fin)

# 혹시 전부 제거된 것이 있는지... -> 하나 있음...
n = 0
for tag in filtered_hashtag:
    if len(tag) == 0:
        n += 1
print(n)

1


In [16]:
# 모델에 있는 단어들만 남겨서 다시 플리 이름이랑 묶어 
id_filtered = hashtags.copy()
id_filtered['tag'] = filtered_hashtag

In [17]:
fasttext.most_similar("사랑한다", topn = 5)

C:\Users\SAMSUNG\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gensim\models\keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


[('웬수야', 0.6226855516433716),
 ('외쳐요', 0.5236533284187317),
 ('사랑하고', 0.5222238302230835),
 ('미안하다', 0.5081436634063721),
 ('사랑해줘', 0.5054067969322205)]

엥........ 모델 상태 왜이래,, 유사 단어 찾기 포기..... ㅠㅠ

In [18]:
# 남은 태그가 하나라면 제거
id_filtered = id_filtered[id_filtered['tag'].apply(lambda x: len(x) > 2)]
print(len(id_filtered))
# 1024 -> 994 로 줄어듦 
id_filtered.head(3)

994


,name,tag,id
0,bg_1,"[여름, 더워, 시원, 청량, 플리, 케이팝, kpop]",1
1,bg_2,"[팝송, 케이팝, 매장, 카페, 식당, 브금, 노래, 휴식, 신나는, 일]",2
2,bg_3,"[아침, 나른, 팝송, 플리, 노래, 산책, 침대, 잔잔, 휴식]",3


In [19]:
# 기존 태그 임베딩, 벡터 평균 함수 
def vector_and_average(input_lists):
    avg_vectors = []
    for list in input_lists:
        vectors = [fasttext[word] for word in list]
        avg = sum(vectors) / len(vectors)
        avg_vectors.append(avg)
    return avg_vectors 

# 새로운 태그 임베딩 함수 
def new_tag_vector(input):
    vectors = [fasttext[word] for word in input]
    avg_vectors = sum(vectors) / len(vectors)
    return avg_vectors

# 태그 유사도 계산
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

# 가장 유사한 플리 출력 
def similar_things(input, original, num): # original : 비교대상 친구들 벡터화하기 전, 단어 형태 
    new_vector = new_tag_vector(input)
    original_vectors = vector_and_average(original)
    # 유사도 계산
    similarities = []
    for i in range(len(original)):
        similarities.append(cos_sim(new_vector, original_vectors[i]))
    # 유사 해시태그 
    top_index = [idx for idx,_ in nlargest(num, enumerate(similarities), key = lambda x:x[1])]
    top_tags = [original[idx] for idx in top_index]
    return top_index, top_tags 

In [21]:
# testing
new_tag = ['신남', '댄스', '드라이브', '여름', '더워', '흥분']

similar_things(new_tag, id_filtered['tag'], 3)

([161, 657, 710],
 [['여름', '남자아이돌', '댄스', '드라이브', '신나는', '기분전환', '스트레스', '운동', '새벽', '바다'],
  ['퇴폐적인', '감각적인', '술', '밤', '저녁', '새벽', '라운지', '칵테일', 'bar', '기분전환'],
  ['가을', '감성', '플리', '기타', '인디', '발라드']])

##### 트랙 2) 음악 취향 필터링, 가사 태그 기반 노래 찾기

결과물 - filtered_songs_nona

In [22]:
# 키워드, 감정 없는 행 제거 (NaN 제거)
songs_tag_nona = songs_tag[~songs_tag['키워드'].isna()]

In [23]:
# 모델에 없는 단어 제외
keywords = songs_tag_nona.iloc[:,6].tolist()
feelings = songs_tag_nona.iloc[:,7].tolist()

filtered_keywords = delete_missing_tags(keywords)
filtered_feelings = delete_missing_tags(feelings)

In [24]:
# 전부 다 없어진 애들이 존재하나? 
n = 0
for tag in filtered_keywords:
    if len(tag) == 0:
        n += 1
print(n)

n = 0
for tag in filtered_feelings:
    if len(tag) == 0:
        n += 1
print(n)     

315
39


어어.... 싹 지워져버린 게 좀 있다만..... 어쩔 수 없지 뭐 일단 진행해야지...

In [25]:
filtered_songs = songs_tag_nona.copy()
filtered_songs['키워드'] = filtered_keywords
filtered_songs['감정'] = filtered_feelings

# 키워드, 감정 합쳐서 tags 정의
filtered_songs['tags'] = filtered_songs['키워드'] + filtered_songs['감정']

In [26]:
# 싹 지워져버린 친구들 제외
filtered_songs_nona = filtered_songs[filtered_songs.iloc[:,8].apply(lambda x: len(x) != 0)]
print(len(filtered_songs_nona))
filtered_songs_nona.head(3)

24083


,제목,가수,가사,발매년도,장르,좋아요 수,키워드,감정,tags
0,I Don't Think That I Like Her,Charlie Puth,Get her name and get her number Find out all o...,2022,POP,214990,"[그녀, 마음, 별]","[슬픔, 후회, 실망]","[그녀, 마음, 별, 슬픔, 후회, 실망]"
1,Dangerously,Charlie Puth,This is gonna hurt but I blame myself first ca...,2016,POP,217897,"[위험, 사랑, 진실]","[슬픔, 후회, 집착]","[위험, 사랑, 진실, 슬픔, 후회, 집착]"
2,STAY,"The Kid LAROI, Justin Bieber",I do the same thing I told you that I never wo...,2021,POP,293599,"[당신, 변화, 머무름]","[그리움, 후회, 불안]","[당신, 변화, 머무름, 그리움, 후회, 불안]"


### 최종 !!!

In [27]:
#### 일기 태그 ####

# ! pip install openai
# ! pip install python-dotenv

import os
from openai import OpenAI
from dotenv import load_dotenv
import ast

# API 
def get_api():
    load_dotenv()
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    return OPENAI_API_KEY

api = get_api()
client = OpenAI(api_key = api)

# Keywords
def get_keywords(input_text):

    prompt = f"""
    You are a helpful asssistant. 
    Extract **four emotions** and **four keyword** in **Korean** from the following text : {input_text}.
    Make sure all the answers are **nouns**.
    Follow the given format : 
    {["emotion1", "emotion2", "emotion3", "emotion4", "keyword1", "keyword2", "keyword3", "keyword4"]}
    """

    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        # max_tokens = 50,
        # temperature = 0.3
    )

    keywords = response.choices[0].message.content
    return keywords 

####################################################################################
input_text = """
방학이 벌써 반이나 지나가버렸어. 
오랫동안 기대하고 꿈꿨던 방학이었는데.. 벌써 반이나 갔다니 너무 아쉬워ㅠㅠ
그래도 남은 절반을 신나고 행복하게 즐겨볼 테야!
이제 연휴니까 푹 쉬고 열심히 공부도 해야지!
"""
#####################################################################################

diary_keywords = get_keywords(input_text)

diary_tags = ast.literal_eval(diary_keywords)
print(diary_tags)

['기대', '꿈', '아쉬움', '행복', '방학', '절반', '신나', '공부']


#### 시도 2

In [29]:
# 트랙 1

# 일기 태그와 모델...
filtered_diary_tags = [word for word in diary_tags if word in fasttext]

# 유사한 플레이리스트
idx, _ = similar_things(filtered_diary_tags, id_filtered['tag'],5)
id.iloc[idx,[0,1]]

,name,tag
73,maggul_3,"['#꿈', '#희망', '#응원', '#행복', '#용기', '#위로', '#시작..."
367,mamel_3,"['#초여름', '#여름', '#뮤직비디오', '#뮤비', '#신나는', '#드라이..."
794,celette_19,"['#올드팝', '#낭만', '#사랑', '#휴식', '#여름밤', '#저녁', '..."
934,heartbeat_19,"['#위로', '#힘내', '#아이돌', '#케이팝', '#여돌', '#남돌', '..."
176,starrynight_12,"['#OST', '#감성', '#서정적', '#설렘', '#생각', '#새벽', '..."


In [33]:
# 트랙 2

# 일기 태그와 모델...
filtered_diary_tags = [word for word in diary_tags if word in fasttext]

# 사용자 입력
genre_input = input("어떤 장르의 음악을 좋아하시나요? (예: 힙합, 발라드 등): ")
likes_input = input("유명한 곡들이 좋으신가요? (o, x로 답해주세요!): ")
release_input = input("최신 노래를 좋아하시나요? (o, x로 답해주세요!): ")

# 취향 필터링 
def filtering(genre_input, release_input, likes_input):
    # 장르 필터링 
    genre_df = filtered_songs_nona[filtered_songs_nona['장르'].str.contains(genre_input)]

    # 발매년도 필터링 
    if release_input == 'o':
        year_df = genre_df[genre_df['발매년도'] >= 2010]
    elif release_input == 'x':
        year_df = genre_df[genre_df['발매년도'] < 2010]
    else:
        print("o 또는 x로 답변해주세요 :)")

    upper_60 = year_df['좋아요 수'].quantile(0.6) # 60% 지점 
    lower_40 = year_df['좋아요 수'].quantile(0.4) # 40% 지점

    if likes_input == 'o':
        likes_df = year_df[year_df['좋아요 수'] >= upper_60]
    elif likes_input == 'x':
        likes_df = year_df[year_df['좋아요 수'] <= lower_40]
    else:
        print("o 또는 x로 답변해주세요 :)")
    
    if likes_df.empty:
        print("취향에 맞는 곡이 없습니다ㅠㅠ")
    else: 
        sim_indices, sim_tags = similar_things(filtered_diary_tags, likes_df['tags'].tolist(), 15)
        return likes_df.iloc[sim_indices,[0,1,6]]

print(filtering(genre_input, likes_input, release_input))

                                             제목                          가수   
3959                            White Christmas              Kelly Clarkson  \
5615                               Come With Me       Surfaces, salem ilese   
24963                                        17              Julia Michaels   
3960                            Candy Cane Lane                         Sia   
7152                 All Night (Feat. Astrid S)           The Vamps, Matoma   
9519                      Christmas Without You                     Ava Max   
11353                           Straight To You            Stephanie Poetri   
1540                                 Lighthouse                 Peder Elias   
13884             Shivers (feat. Jessi & SUNMI)  Ed Sheeran, 제시 (Jessi), 선미   
6975                                     Hailey               Justin Bieber   
7108                                    Skyline                      Khalid   
20479  あの夢をなぞって (그 꿈을 덧그리며, Ano Yumewo Nazotte)     